In [4]:
!nvidia-smi

Thu Jul 25 06:00:46 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.99                 Driver Version: 555.99         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8              3W /   40W |     330MiB /   4096MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:

import ultralytics
print('Ultralytics: ' , ultralytics.checks())

import torch
print('CUDA: ', torch.cuda.is_available())

import torch
print('torch:', torch.__version__)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")



Ultralytics YOLOv8.0.196  Python-3.12.4 torch-2.4.0 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Setup complete  (12 CPUs, 15.7 GB RAM, 222.2/228.2 GB disk)


Ultralytics:  None
CUDA:  True
torch: 2.4.0
Using device: cuda


In [ ]:
import cv2
from ultralytics import YOLO
import torch
import numpy as np 

# Device Setup (Explicit Check):
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load YOLO Model (Ensure Correct Path and Model Exists):
model_path = 'best.pt'  # Adjust if your model file is named differently
model = YOLO(model_path)  # Make sure the model exists at this path
model.to(device)

# Video Capture (Improved Error Handling):
try:
    webcamera = cv2.VideoCapture(0) 
except Exception as e:
    print(f"Error opening webcam: {e}")
    exit()

# Video Loop (with Confidence and IoU Thresholds):
conf_threshold = 0.5   # Adjust confidence threshold for detection
iou_threshold = 0.45  # Adjust IoU (Intersection over Union) threshold

while True:
    success, frame = webcamera.read()
    if not success:
        break

    # Preprocessing:
    # frame = cv2.cvtColor(frame, cv2.q)
    frame = frame.astype(np.float32) / 255.0
    frame_tensor = torch.from_numpy(frame).to(device)
    frame_tensor = frame_tensor.permute(2, 0, 1).unsqueeze(0)

    # Inference:
    results = model.predict(frame_tensor, conf=conf_threshold, iou=iou_threshold)

    # Display Results (Optimized Plotting):
    for result in results:
        annotated_frame = result.plot() 

        # Display count for this result:
        cv2.putText(annotated_frame, f"Count: {len(result.boxes)}", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Show the frame:
        cv2.imshow("Live Camera", annotated_frame)

    # Exit on 'ESC' press:
    if cv2.waitKey(1) == 27:
        break

# Release resources:
webcamera.release()
cv2.destroyAllWindows()